When I'm not even in my harshest

You can still get roasted

'cause Marsh is not Mellow

-- No Love

In [2]:
import spotipy
import pandas as pd
import requests
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.exceptions import SpotifyException
from timeit import default_timer as timer
from datetime import timedelta
import configparser
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.api.types import CategoricalDtype
import configparser
import os

In [3]:
current_dir = os.path.dirname(os.path.abspath("Eminem"))

config_path = os.path.join(current_dir, '..', 'config.ini')

config = configparser.ConfigParser()

config.read(config_path)

client_id = config.get('credentials', 'Client_ID')
client_secret = config.get('credentials', 'Client_Secret')

client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [4]:
def timekeeper(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

In [5]:
from spotipy.oauth2 import SpotifyClientCredentials

# Get credentials from the config file
client_id= '31d85b3f430e45caba7b97174cb75137'
client_secret = '5c6b719bcd8743b0ad1f8cd217ad1e1c'
client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
def collect_spotify_data():
    start_time = timer()
    data = []  


    for i in range(0, 1000, 50):
        try:
            track_results = sp.search(q='artist:Eminem', type='track', limit=50, offset=i)
            for item in track_results['tracks']['items']:
                track_info = {
                    'artist_name': item['artists'][0]['name'],
                    'track_name': item['name'],
                    'track_id': item['id'],
                    'album_name': item['album']['name'],
                    'album_id': item['album']['id'],
                    'release_date': item['album']['release_date'],
                    'popularity': item['popularity'],
                    'explicit': item['explicit']
                }
                
                # Get audio features for track
                audio_features = sp.audio_features(item['id'])[0]
                if audio_features is not None:
                    audio_info = {
                        'danceability': audio_features.get('danceability', float('nan')),
                        'duration_ms': audio_features.get('duration_ms', float('nan')),
                        'energy': audio_features.get('energy', float('nan')),
                        'key': audio_features.get('key', float('nan')),
                        'loudness': audio_features.get('loudness', float('nan')),
                        'mode': audio_features.get('mode', float('nan')),
                        'speechiness': audio_features.get('speechiness', float('nan')),
                        'acousticness': audio_features.get('acousticness', float('nan')),
                        'instrumentalness': audio_features.get('instrumentalness', float('nan')),
                        'liveness': audio_features.get('liveness', float('nan')),
                        'valence': audio_features.get('valence', float('nan')),
                        'tempo': audio_features.get('tempo', float('nan')),
                        'time_signature': audio_features.get('time_signature', float('nan'))
                    }
                    track_info.update(audio_info)
                else:
                    # Handle case where audio features are not available
                    audio_info = {
                        'danceability': float('nan'),
                        'duration_ms': float('nan'),
                        'energy': float('nan'),
                        'key': float('nan'),
                        'loudness': float('nan'),
                        'mode': float('nan'),
                        'speechiness': float('nan'),
                        'acousticness': float('nan'),
                        'instrumentalness': float('nan'),
                        'liveness': float('nan'),
                        'valence': float('nan'),
                        'tempo': float('nan'),
                        'time_signature': float('nan')
                    }
                    track_info.update(audio_info)

                # Get featured artists
                if len(item['artists']) > 1:
                    featured_artists = [artist['name'] for artist in item['artists'][1:]]
                    track_info['featured_artists'] = featured_artists
                else:
                    track_info['featured_artists'] = []

                data.append(track_info)

        except requests.exceptions.ReadTimeout as e:
            print(f"Timeout error: {e}. Retrying in 5 seconds...")
            time.sleep(5)  # Retry after a short delay
        except requests.exceptions.RequestException as e:
            print(f"API error: {e}")
            break  # Break the loop if there's a persistent error

    df_raw = pd.DataFrame(data)
    end_time = timer()
    elapsed_time = int(end_time - start_time)
    print(f"Elapsed time:", timekeeper(elapsed_time))
    return df_raw

# Collecting data from Spotify API 
df_spotify = collect_spotify_data()


Elapsed time: 4 minutes, 12 seconds


In [13]:
df_spotify

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Eminem,Without Me,7lQ8MOhq6IN2w8EYcFNSUk,The Eminem Show,2cWBwpqMsDJC1ZUwz813lo,2002-05-26,86,True,0.908,290320,...,-2.827,1,0.0738,0.00286,0.000000,0.2370,0.662,112.238,4,[]
1,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,The Marshall Mathers LP,6t7956yu5zYf5A829XRiHC,2000-05-23,84,True,0.949,284200,...,-4.244,0,0.0572,0.03020,0.000000,0.0454,0.760,104.504,4,[]
2,Eminem,Till I Collapse,4xkOaSrkexMciUUogZKVTS,The Eminem Show,2cWBwpqMsDJC1ZUwz813lo,2002-05-26,82,True,0.548,297787,...,-3.237,1,0.1860,0.06220,0.000000,0.0816,0.100,171.447,4,[Nate Dogg]
3,Eminem,Mockingbird,561jH07mF1jHuk7KlaeF0s,Encore (Deluxe Version),1kTlYbs28MXw7hwO0NLYif,2004-11-12,85,True,0.637,250760,...,-3.798,1,0.2660,0.20900,0.000000,0.1560,0.254,84.039,4,[]
4,Akon,Smack That,2kQuhkFX7uSVepCD3h29g5,Konvicted,6kzoWb4UzvKYgbDfAwgaFq,2006-01-01,83,True,0.939,212360,...,-5.171,0,0.0471,0.32300,0.000000,0.1070,0.925,118.978,4,[Eminem]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Eminem,Still Don't Give,2NPIFzhYie0mIAjxr5kw6a,The Slim Shady LP,2VFlCEu1NPLIbWjIw1lmjJ,1999-02-23,22,False,0.635,250360,...,-2.914,0,0.4120,0.15500,0.000000,0.3640,0.665,89.829,4,[]
996,Eminem,Bad Guy,2D3V1b3N0ocScCEbajPPvI,The Marshall Mathers LP2 (Deluxe),6Vn28iXHIwxmvh4Va0hSoe,2013-11-05,17,False,0.500,434077,...,-4.014,1,0.5190,0.05960,0.000000,0.2500,0.329,83.449,4,[]
997,Eminem,Role Model,4NQcaKgDJzxcUT4Ap44d9s,The Slim Shady LP,2VFlCEu1NPLIbWjIw1lmjJ,1999-02-23,24,True,0.853,210400,...,-4.311,1,0.2550,0.02040,0.000008,0.0983,0.824,91.911,4,[]
998,Eminem,Little Engine,5X1A8lPx3YRTNCV83NG57o,Music To Be Murdered By,2LWLgGY15e9im0KUaASnFa,2020-01-17,20,False,0.763,177293,...,-4.192,1,0.2390,0.02220,0.000000,0.0404,0.725,155.079,4,[]


In [14]:
shady = df_spotify[df_spotify['artist_name']=="Eminem"].copy()
shady

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Eminem,Without Me,7lQ8MOhq6IN2w8EYcFNSUk,The Eminem Show,2cWBwpqMsDJC1ZUwz813lo,2002-05-26,86,True,0.908,290320,...,-2.827,1,0.0738,0.00286,0.000000,0.2370,0.6620,112.238,4,[]
1,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,The Marshall Mathers LP,6t7956yu5zYf5A829XRiHC,2000-05-23,84,True,0.949,284200,...,-4.244,0,0.0572,0.03020,0.000000,0.0454,0.7600,104.504,4,[]
2,Eminem,Till I Collapse,4xkOaSrkexMciUUogZKVTS,The Eminem Show,2cWBwpqMsDJC1ZUwz813lo,2002-05-26,82,True,0.548,297787,...,-3.237,1,0.1860,0.06220,0.000000,0.0816,0.1000,171.447,4,[Nate Dogg]
3,Eminem,Mockingbird,561jH07mF1jHuk7KlaeF0s,Encore (Deluxe Version),1kTlYbs28MXw7hwO0NLYif,2004-11-12,85,True,0.637,250760,...,-3.798,1,0.2660,0.20900,0.000000,0.1560,0.2540,84.039,4,[]
5,Eminem,Lose Yourself,5Z01UMMf7V1o0MzF86s6WJ,Curtain Call: The Hits (Deluxe Edition),5qENHeCSlwWpEzb25peRmQ,2005-12-06,76,True,0.692,326467,...,-4.565,1,0.2570,0.00868,0.000423,0.3590,0.0612,171.374,4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Eminem,Still Don't Give,2NPIFzhYie0mIAjxr5kw6a,The Slim Shady LP,2VFlCEu1NPLIbWjIw1lmjJ,1999-02-23,22,False,0.635,250360,...,-2.914,0,0.4120,0.15500,0.000000,0.3640,0.6650,89.829,4,[]
996,Eminem,Bad Guy,2D3V1b3N0ocScCEbajPPvI,The Marshall Mathers LP2 (Deluxe),6Vn28iXHIwxmvh4Va0hSoe,2013-11-05,17,False,0.500,434077,...,-4.014,1,0.5190,0.05960,0.000000,0.2500,0.3290,83.449,4,[]
997,Eminem,Role Model,4NQcaKgDJzxcUT4Ap44d9s,The Slim Shady LP,2VFlCEu1NPLIbWjIw1lmjJ,1999-02-23,24,True,0.853,210400,...,-4.311,1,0.2550,0.02040,0.000008,0.0983,0.8240,91.911,4,[]
998,Eminem,Little Engine,5X1A8lPx3YRTNCV83NG57o,Music To Be Murdered By,2LWLgGY15e9im0KUaASnFa,2020-01-17,20,False,0.763,177293,...,-4.192,1,0.2390,0.02220,0.000000,0.0404,0.7250,155.079,4,[]


In [15]:
shady['album_name'].unique()

array(['The Eminem Show', 'The Marshall Mathers LP',
       'Encore (Deluxe Version)',
       'Curtain Call: The Hits (Deluxe Edition)',
       'Music To Be Murdered By', 'Houdini', 'Recovery',
       'The Death of Slim Shady (Coup De Grâce)',
       'The Marshall Mathers LP2', 'The Slim Shady LP',
       'The Marshall Mathers LP2 (Deluxe)', 'Kamikaze', 'Killshot',
       'Tobey (feat. Big Sean and BabyTron)', 'Revival',
       'Relapse: Refill',
       'The Death of Slim Shady (Coup De Grâce): Expanded Mourner’s Edition',
       'Killer (feat. Jack Harlow & Cordae) [Remix]', "When I'm Gone",
       'Curtain Call: The Hits', 'SHADYXV',
       'Venom (Music From The Motion Picture)', 'Cradle 2 The Grave',
       'The Eminem Show (Expanded Edition)',
       'Music To Be Murdered By - Side B (Deluxe Edition)',
       'Eminem Presents The Re-Up',
       '8 Mile (Music From And Inspired By The Motion Picture)',
       'From The D 2 The LBC (with Snoop Dogg)',
       'Somebody Save Me (feat.

In [ ]:
shady[shady['album_name']=='The Marshall Mathers LP2 (Deluxe)']

In [ ]:
albums = [
    'The Slim Shady LP',
    'The Marshall Mathers LP',
    'The Eminem Show',
    'Encore',
    'Relapse', 
    'Recovery',
    'The Marshall Mathers LP2', 
    'Revival',
    'Kamikaze',
    'Music To Be Murdered By', 
    'The Death of Slim Shady (Coup De Grâce)'
]

deluxe_editions = [
    'The Death of Slim Shady (Coup De Grâce): Expanded Mourner’s Edition',
    'The Marshall Mathers LP2 (Deluxe)', 
    'The Eminem Show (Expanded Edition)',
    'The Slim Shady LP (Expanded Edition)',
    'Encore (Deluxe Version)',
    'Recovery (Deluxe Edition)',
    'Relapse [Deluxe]'
]
soundtrack = [
    'Southpaw (Music From And Inspired By The Motion Picture)',
    '8 Mile (Music From And Inspired By The Motion Picture)'
]

collaborative_albums = [
    'SHADYXV',
    'Eminem Presents The Re-Up'
]

reissued_albums = [
    'Relapse: Refill',
    'Music To Be Murdered By - Side B (Deluxe Edition)'
]

compilation_albums = [
    'Curtain Call: The Hits (Deluxe Edition)',
    'Curtain Call 2'
]

singles = shady['track_name']==shady['album_name']
album = shady['album_name'].isin(albums)
deluxe = shady['album_name'].isin(deluxe_editions)



conditions = [singles, album]
choices = ['Single', 'Studio Album']
shady['type'] = np.select(conditions, choices, default='')